<h1>Import necessary libraries

In [284]:
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as psql

<h1>Connect to DB

In [14]:
conn = pg.connect(" dbname='dev' host='njoy-testing-env.c2ag7j9q24ns.us-east-1.rds.amazonaws.com' user='reader' port='5416' password='MbmeL8ku6x'")
df = psql.read_sql('SELECT count(1) from public.orders', conn)
df.head()

,count
0,201570


In [15]:
df = psql.read_sql('SELECT * from public.orders', conn)

<h1>Preview of DataFrame

In [44]:
df.head(5)

,order_id,customer_id,created_at,updated_at,closed_at,processed_at,status,number,total_price,total_less_tax,total_tax,payment_status,closed_hour
0,8.974965e+11,230824c2,2019-03-13 18:14:55,2019-03-13 22:01:40,2019-03-13 22:01:39,2019-03-13 18:14:55.000000,fulfilled,1029702,18.28,17.0,1.28,paid,22.0
1,8.975937e+11,7100545b,2019-03-13 19:33:50,2019-03-13 23:01:21,2019-03-13 23:01:21,2019-03-13 19:33:50.000000,fulfilled,1029768,9.00,9.0,0.00,paid,23.0
2,8.975785e+11,3de1dc29,2019-03-13 19:20:05,2019-03-13 23:01:33,2019-03-13 23:01:33,2019-03-13 19:20:05.000000,fulfilled,1029757,18.02,17.0,1.02,paid,23.0
3,8.976117e+11,cb5e4189,2019-03-13 19:50:09,2019-03-13 23:01:12,2019-03-13 23:01:11,2019-03-13 19:50:09.000000,fulfilled,1029780,9.63,9.0,0.63,paid,23.0
4,8.976406e+11,579c3141,2019-03-13 20:17:01,2019-03-14 00:01:23,2019-03-14 00:01:23,2019-03-13 20:17:01.000000,fulfilled,1029803,9.26,9.0,0.26,paid,0.0


In [73]:
df.shape

(201570, 13)

<h1>Query 1</h1>
<h6>We need basic insight into hourly sales performance. We'd like a query that shows the following metrics by hour: <br>
revenue (total_price less total_tax)<br>
total number of orders<br>
number of orders placed by a customer who has previously placed an order<br>

In [156]:
#create column to seperate closing hour of orders
df['closed_hour'] = df['closed_at'].dt.hour

In [201]:
hourly_orders = df.groupby('closed_hour')['total_less_tax'].agg(['sum','count']).reset_index()

In [86]:
customer_counts = df.groupby('customer_id').count().reset_index().sort_values('order_id', ascending = False)

In [93]:
#there are 181,105 total customers in the dataframe
customer_counts.shape

(181105, 13)

In [165]:
#there are 19,339 returning customers
returning_customers = customer_counts.loc[customer_counts['order_id'] > 1][['customer_id', 'order_id']]

In [161]:
def find_hours_of_returning_customers():
    returning_customer_hours = {}
    for customer in returning_customers.customer_id.to_list():
        order_hours = df.loc[df['customer_id'] == customer].closed_hour.to_list()
        for i in order_hours:
            try:
                if int(i) not in returning_customer_hours.keys():
                    returning_customer_hours.update({i: 1})
                else:
                    returning_customer_hours[i] +=1
            except:
                pass
    return returning_customer_hours

In [181]:
total_returning_per_hour = find_hours_of_returning_customers()

In [191]:
returning_df = pd.DataFrame(total_returning_per_hour, index = ['number']).transpose().reset_index()
returning_df_sorted = returning_df.sort_values('index')

In [210]:
final_orders = pd.merge(hourly_orders, returning_df_sorted, left_on='closed_hour', right_on='index')
final_orders = final_orders.drop(['index'], axis = 1)
final_orders.columns = ['hour', 'total_revenue', 'num_orders', 'num_returning_customers']

In [211]:
final_orders

,hour,total_revenue,num_orders,num_returning_customers
0,0.0,54508.00,4481,585
1,1.0,62859.27,5495,706
2,13.0,137.00,6,3
3,14.0,43279.00,3973,241
4,15.0,18872.50,1838,148
5,16.0,185631.40,18287,2881
6,17.0,438159.20,45940,10063
7,18.0,522385.90,54200,12533
8,19.0,233999.00,23461,4219
9,20.0,152570.80,14325,3369


<h1>Query 2</h1>
<h6>Upon further inspection, it has been noticed that there are a number of hours for which there are no orders. It's not clear why, so we'd like a query that lists all hours for which there are no orders.


In [219]:
hours_with_orders = df.groupby(df.closed_hour).count().reset_index()['closed_hour'].to_list()

In [220]:
all_hours = list(range(0,24))

In [223]:
hours_without_orders = []
for hour in all_hours:
    if hour not in hours_with_orders:
        hours_without_orders.append(hour)

In [242]:
no_orders = pd.DataFrame(hours_without_orders)

In [245]:
no_orders.columns = ['hour']

In [257]:
no_orders

,hour
0,2
1,3
2,4
3,5
4,6
5,7
6,8
7,9
8,10
9,11


<h1>Query 3</h1>
<h6>Your query from #1 may not have accounted for hours for which there are no orders. Copy your query from #1 and ammend the query to show "0" for all metrics when there is no data for a given hour.

In [253]:
all_hourly_orders = pd.concat([final_orders, no_orders], join = 'outer')

/Users/austinkrause/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [255]:
all_hourly_orders = all_hourly_orders.sort_values('hour')

In [256]:
all_hourly_orders

,hour,num_orders,num_returning_customers,total_revenue
0,0.0,4481.0,585.0,54508.00
1,1.0,5495.0,706.0,62859.27
0,2.0,NaN,NaN,NaN
1,3.0,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN
3,5.0,NaN,NaN,NaN
4,6.0,NaN,NaN,NaN
5,7.0,NaN,NaN,NaN
6,8.0,NaN,NaN,NaN
7,9.0,NaN,NaN,NaN


<h1>Query 4</h1>
<h6>We'd like to know the difference in revenue from customers who have completed one order versus customers who have completed more than one order. Write a query that shows the total revenue (total_price less total_tax) for each of these two groups: New Customers and Returning Customers.

In [267]:
customer_totals = df.groupby('customer_id')['total_less_tax'].agg(['sum', 'count']).reset_index()

In [268]:
customer_totals = customer_totals.sort_values('count')

In [270]:
customer_totals.columns = ['customer_id', 'total_revenue', 'num_orders']

In [281]:
new_customers = customer_totals.loc[customer_totals['num_orders'] == 1]
returning_customers = customer_totals.loc[customer_totals['num_orders'] != 1]

In [282]:
new_customers['total_revenue'].sum()

1634850.9699999997

In [283]:
returning_customers['total_revenue'].sum()

397131.3

<h1>Query 5</h1>
<h6>We'd like a rudimentary estimate of when a customer might place another order and the next order's value, based on the time between their last two orders. For all customers that have completed more than one order, write a query that shows the customer_id, last_order_date and next_projected_order_date, and next_projected_order_value where next_projected_order_date is their last order plus the amount of time between their last two orders and next_projected_order_value is the average of their last two orders' total_price less total_tax.

In [286]:
returning_list = returning_customers.customer_id.to_list()

In [291]:
returning_full_df = df[df['customer_id'].isin(returning_list)]

In [390]:
returning_full_df = returning_full_df.sort_values(['customer_id', 'closed_at'])

In [391]:
returning_full_df

,order_id,customer_id,created_at,updated_at,closed_at,processed_at,status,number,total_price,total_less_tax,total_tax,payment_status,closed_hour
72589,8.561098e+11,0005aaf0,2019-01-05 11:33:56,2019-01-07 17:06:09,2019-01-07 17:06:09,2019-01-05 11:33:56,fulfilled,875783,9.00,9.0,0.00,paid,17.0
72550,8.561096e+11,0005aaf0,2019-01-05 11:33:46,2019-01-07 17:06:10,2019-01-07 17:06:10,2019-01-05 11:33:46,fulfilled,875781,9.00,9.0,0.00,paid,17.0
17935,8.921384e+11,0005aaf0,2019-03-09 11:28:13,2019-03-11 16:20:44,2019-03-11 16:20:44,2019-03-09 11:28:13.000000,fulfilled,1008794,9.00,9.0,0.00,paid,16.0
12222,8.921383e+11,0005aaf0,2019-03-09 11:27:59,2019-03-11 16:20:46,2019-03-11 16:20:46,2019-03-09 11:27:59.000000,fulfilled,1008792,9.00,9.0,0.00,paid,16.0
137103,8.622300e+11,000b5078,2019-02-14 11:16:30,2019-02-14 18:23:00,2019-02-14 18:23:00,2019-02-14 11:16:30.000000,fulfilled,895204,9.54,9.0,0.54,paid,18.0
11247,8.981644e+11,000b5078,2019-03-14 08:13:57,2019-03-14 17:38:56,2019-03-14 17:38:56,2019-03-14 08:13:57.000000,fulfilled,1030834,9.54,9.0,0.54,paid,17.0
24782,8.622481e+11,0011f216,2019-02-14 11:37:04,2019-02-14 18:20:16,2019-01-10 18:20:16,2019-01-10 11:37:04,fulfilled,895417,9.00,9.0,0.00,paid,18.0
16501,8.981538e+11,0011f216,2019-03-14 07:59:21,2019-03-14 17:41:06,2019-03-14 17:41:06,2019-03-14 07:59:21.000000,fulfilled,1030676,9.00,9.0,0.00,paid,17.0
52255,8.561230e+11,0015e03c,2019-01-05 11:54:27,2019-01-07 17:03:49,2019-01-07 17:03:49,2019-01-05 11:54:27,fulfilled,875950,9.70,9.0,0.70,paid,17.0
4411,8.921276e+11,0015e03c,2019-03-09 11:12:58,2019-03-11 16:22:39,2019-03-11 16:22:39,2019-03-09 11:12:58.000000,fulfilled,1008648,9.70,9.0,0.70,paid,16.0


In [299]:
#returning_full_df.groupby('customer_id')['closed_at'].apply(lambda x: (x-x[:-1]).astype('m8[h]').sum()).reset_index()

In [319]:
returning_full_df.loc[returning_full_df['customer_id'] == '0005aaf0']['closed_at'] - returning_full_df.loc[returning_full_df['customer_id'] == '0005aaf0']['closed_at']

72550   0 days
17935   0 days
12222   0 days
72589   0 days
Name: closed_at, dtype: timedelta64[ns]

In [312]:
test_return = returning_full_df.loc[returning_full_df['customer_id'] == returning_list[0]]
test_return

,order_id,customer_id,created_at,updated_at,closed_at,processed_at,status,number,total_price,total_less_tax,total_tax,payment_status,closed_hour
176227,8.855546e+11,7147f69d,2019-03-04 08:56:10,2019-03-04 20:05:04,2019-03-04 20:05:04,2019-03-04 08:56:10.000000,fulfilled,983086,9.63,9.0,0.63,paid,20.0
45863,8.499658e+11,7147f69d,2018-12-31 09:47:07,2018-12-31 18:38:22,2018-12-31 18:38:22,2018-12-31 09:47:07,fulfilled,854958,9.63,9.0,0.63,paid,18.0


In [316]:
test_return['closed_at'].iloc[-2] - test_return['closed_at'].iloc[-1]

Timedelta('63 days 01:26:42')

In [318]:
next_purchase_date = test_return['closed_at'].iloc[-2] + (test_return['closed_at'].iloc[-2] - test_return['closed_at'].iloc[-1])

Timestamp('2019-05-06 21:31:46')

In [362]:
(test_return['total_less_tax'].iloc[-2] + test_return['total_less_tax'].iloc[-1])/2

9.0

In [398]:
def project_next_purchase(list_of_customers):
    projected_purchases = []
    customer_index = 0
    
    for i in list_of_customers:
        
        customer_df = returning_full_df.loc[returning_full_df['customer_id'] == i]

        next_purchase_info = {}
        next_purchase_info['customer_id'] = i
        next_purchase_info['last_order_date'] = customer_df['closed_at'].iloc[-1]
        next_purchase_info['next_projected_order_date'] = customer_df['closed_at'].iloc[-1] + (customer_df['closed_at'].iloc[-1] - customer_df['closed_at'].iloc[-2])
        next_purchase_info['next_projected_order_value'] = (customer_df['total_less_tax'].iloc[-2] + customer_df['total_less_tax'].iloc[-1])/2
        projected_purchases.append(next_purchase_info)
        customer_index += 1
        
    return projected_purchases

In [399]:
project_next_purchase(returning_list)

[{'customer_id': '7147f69d',
  'last_order_date': Timestamp('2019-03-04 20:05:04'),
  'next_projected_order_date': Timestamp('2019-05-06 21:31:46'),
  'next_projected_order_value': 9.0},
 {'customer_id': '00286940',
  'last_order_date': Timestamp('2019-03-11 18:31:57'),
  'next_projected_order_date': Timestamp('2019-03-22 17:10:30'),
  'next_projected_order_value': 9.0},
 {'customer_id': '87885c02',
  'last_order_date': Timestamp('2019-03-04 18:26:55'),
  'next_projected_order_date': Timestamp('2019-05-06 18:47:29'),
  'next_projected_order_value': 9.0},
 {'customer_id': 'f5653a6d',
  'last_order_date': NaT,
  'next_projected_order_date': NaT,
  'next_projected_order_value': 9.0},
 {'customer_id': '78e87a63',
  'last_order_date': Timestamp('2019-01-01 00:02:46'),
  'next_projected_order_date': Timestamp('2019-01-01 03:51:38'),
  'next_projected_order_value': 8.5},
 {'customer_id': '72436c76',
  'last_order_date': Timestamp('2019-03-06 20:02:13'),
  'next_projected_order_date': Timestam

In [400]:
projected_purchases

[{'customer_id': '7147f69d',
  'last_order_date': Timestamp('2018-12-31 18:38:22'),
  'next_projected_order_date': Timestamp('2019-05-06 21:31:46'),
  'next_projected_order_value': 9.0},
 {'customer_id': '00286940',
  'last_order_date': Timestamp('2019-02-28 19:53:24'),
  'next_projected_order_date': Timestamp('2019-03-22 17:10:30'),
  'next_projected_order_value': 9.0},
 {'customer_id': '87885c02',
  'last_order_date': Timestamp('2018-12-31 18:06:21'),
  'next_projected_order_date': Timestamp('2019-05-06 18:47:29'),
  'next_projected_order_value': 9.0},
 {'customer_id': 'f5653a6d',
  'last_order_date': Timestamp('2019-02-15 17:31:55'),
  'next_projected_order_date': NaT,
  'next_projected_order_value': 9.0},
 {'customer_id': '78e87a63',
  'last_order_date': Timestamp('2019-01-01 00:02:46'),
  'next_projected_order_date': Timestamp('2018-12-31 16:25:02'),
  'next_projected_order_value': 8.5},
 {'customer_id': '72436c76',
  'last_order_date': Timestamp('2019-03-06 20:02:13'),
  'next_pr

In [401]:
projected_purchases_df = pd.DataFrame.from_dict(projected_purchases)

In [402]:
projected_purchases_df

,customer_id,last_order_date,next_projected_order_date,next_projected_order_value
0,7147f69d,2018-12-31 18:38:22,2019-05-06 21:31:46,9.0
1,00286940,2019-02-28 19:53:24,2019-03-22 17:10:30,9.0
2,87885c02,2018-12-31 18:06:21,2019-05-06 18:47:29,9.0
3,f5653a6d,2019-02-15 17:31:55,NaT,9.0
4,78e87a63,2019-01-01 00:02:46,2018-12-31 16:25:02,8.5
5,72436c76,2019-03-06 20:02:13,2018-10-31 20:46:53,9.0
6,0745389a,2019-03-06 19:03:00,2018-10-31 21:20:30,9.0
7,ffcf3677,2018-12-31 17:52:18,2019-05-06 19:47:10,9.0
8,8786ddf4,2019-02-13 18:19:19,2019-04-12 16:26:27,9.0
9,878940f1,2019-02-11 17:26:52,2019-04-08 16:50:32,13.5
